In [1]:
pip install pyamaze

Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List, Tuple
from random import randint, random
from pyamaze import maze, agent, COLOR
import csv

In [3]:
# Maze size
Num_rows = 7
Num_col = 20
GOAL = (7,19)
START = (1,1)

# Algorithm Parameter
POPULATION_SIZE = 2000
MAX_GENERATIONS = 5000
TARGET_FITNESS = 75
MUTATION_PROBABILITY = 0.06
CROSSOVER_RATE = 0.7
MUTATION_LENGTH = 15

In [4]:
# list of Individuals will be collected in Population list
Genome = List[int]
Population = List[Genome]

In [5]:
# Load maze from CSV
def load_maze_from_csv(filename: str) -> dict:
    maze_map = {}
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:
            if len(row) < 5:
                continue
            cell = eval(row[0])
            maze_map[cell] = {'E': int(row[1]), 'W': int(row[2]), 'N': int(row[3]), 'S': int(row[4])}
    return maze_map


In [6]:
maze_map = load_maze_from_csv('myMaze.csv')
maze_map

{(1, 1): {'E': 1, 'W': 0, 'N': 0, 'S': 0},
 (2, 1): {'E': 1, 'W': 0, 'N': 0, 'S': 1},
 (3, 1): {'E': 1, 'W': 0, 'N': 1, 'S': 0},
 (4, 1): {'E': 1, 'W': 0, 'N': 0, 'S': 1},
 (5, 1): {'E': 0, 'W': 0, 'N': 1, 'S': 1},
 (6, 1): {'E': 0, 'W': 0, 'N': 1, 'S': 1},
 (7, 1): {'E': 0, 'W': 0, 'N': 1, 'S': 0},
 (1, 2): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (2, 2): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (3, 2): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (4, 2): {'E': 0, 'W': 1, 'N': 0, 'S': 1},
 (5, 2): {'E': 1, 'W': 0, 'N': 1, 'S': 0},
 (6, 2): {'E': 1, 'W': 0, 'N': 0, 'S': 1},
 (7, 2): {'E': 1, 'W': 0, 'N': 1, 'S': 0},
 (1, 3): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (2, 3): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (3, 3): {'E': 0, 'W': 1, 'N': 0, 'S': 1},
 (4, 3): {'E': 1, 'W': 0, 'N': 1, 'S': 0},
 (5, 3): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (6, 3): {'E': 1, 'W': 1, 'N': 0, 'S': 0},
 (7, 3): {'E': 0, 'W': 1, 'N': 0, 'S': 0},
 (1, 4): {'E': 1, 'W': 1, 'N': 0, 'S': 1},
 (2, 4): {'E': 0, 'W': 1, 'N': 1, 'S': 0},
 (3, 4): {'

In [7]:
# Create maze with pyamaze
m = maze(Num_rows, Num_col)
m.CreateMaze(GOAL[0], GOAL[1], loadMaze="myMaze.csv", theme=COLOR.dark)

In [8]:
# Generate a random genome
def generate_genome(num_col: int, num_rows: int) -> Genome:
    genome = [randint(1, num_rows - 1) for _ in range(num_col)]  # Generate rows between 1 and num_rows-1
    genome.insert(0, START[0])  # Ensure start point
    genome.append(GOAL[0])  # Ensure end point
    return genome

# Generate initial population
def generate_population(size: int, num_col: int, num_rows: int) -> Population:
    return [generate_genome(num_col, num_rows) for _ in range(size)]

# Fitness function
def fitness_function(population, maze):
    fitness_scores = []
    infeasible_steps = []
    goal = GOAL

    for genome in population:
        path = path_from_genome(genome, Num_col)
        infsteps = 0

        for step in range(len(path) - 1):
            current = path[step]
            next_step = path[step + 1]

            # Ensure both current and next step are within the maze bounds
            if not (1 <= current[0] <= Num_rows and 1 <= current[1] <= Num_col):
                infsteps += 1
                continue
            if not (1 <= next_step[0] <= Num_rows and 1 <= next_step[1] <= Num_col):
                infsteps += 1
                continue

            # Check feasibility of steps based on maze walls and boundaries
            if current[0] > next_step[0]:  # Moving North
                if maze_map[current]['N'] == 0:  # Wall to the North
                    infsteps += 1
            elif current[0] < next_step[0]:  # Moving South
                if maze_map[current]['S'] == 0:  # Wall to the South
                    infsteps += 1
            elif current[1] > next_step[1]:  # Moving West
                if maze_map[current]['W'] == 0:  # Wall to the West
                    infsteps += 1
            elif current[1] < next_step[1]:  # Moving East
                if maze_map[current]['E'] == 0:  # Wall to the East
                    infsteps += 1


        # Check the final step to the goal
        last_point = path[-1]
        if last_point != goal:
            # Check if we can move from the last point to the goal
            if last_point[0] > goal[0]:  # Moving North to goal
                if maze_map[last_point]['N'] == 0:  # Wall to the North
                    infsteps += 1
            elif last_point[0] < goal[0]:  # Moving South to goal
                if maze_map[last_point]['S'] == 0:  # Wall to the South
                    infsteps += 1
            elif last_point[1] > goal[1]:  # Moving West to goal
                if maze_map[last_point]['W'] == 0:  # Wall to the West
                    infsteps += 1
            elif last_point[1] < goal[1]:  # Moving East to goal
                if maze_map[last_point]['E'] == 0:  # Wall to the East
                    infsteps += 1

        # Penalize infeasible steps and longer paths
        path_length = len(path)

        # Manhattan distance from the last point to the goal
        if path:
            last_point = path[-1]
            distance_to_goal = abs(last_point[0] - goal[0]) + abs(last_point[1] - goal[1])
        else:
            distance_to_goal = float('inf')  # Path doesn't exist, large penalty

        # Fitness: fewer infeasible steps is better, shorter distance to goal is better
        fitness = 100 - infsteps * 5 - path_length * 0.1 - distance_to_goal * 2
        fitness_scores.append(fitness)
        infeasible_steps.append(infsteps)


    return fitness_scores, infeasible_steps

# Crossover with mod7
def mod7_crossover(parent1: Genome, parent2: Genome) -> Tuple[Genome, Genome]:
    if random() <= CROSSOVER_RATE: # CROSSOVER_RATE of chance for crossover
        child1 = []
        child2 = []
        for gene1, gene2 in zip(parent1, parent2):
            child1.append(((gene1 + gene2) % 7) + 1)  # Sum of genes mod 7
            child2.append((abs(gene1 - gene2) % 7)+ 1)  # Absolute difference of genes mod 7
    else:
        # No crossover, return parents as children
        child1 = parent1[:]
        child2 = parent2[:]

    child1[0],child2[0],child1[-1],child2[-1] = START[0], START[0], GOAL[0], GOAL[0]

    return child1, child2

# Segmented Mutation with Random Segment Length
def segmented_mutation(genome: Genome, mutation_probability: float = MUTATION_PROBABILITY) -> Genome:
    num_genes = len(genome)
    start_idx = 1  # Start after the first gene (START point)

    while start_idx < num_genes - 1:  # Ensure we don't mutate START or GOAL
        # Randomly determine the segment length (between 1 and 14)
        segment_length = randint(1, MUTATION_LENGTH)

        # Determine the end index of the current segment, ensuring it doesn't go beyond the genome size
        end_idx = min(start_idx + segment_length, num_genes - 1)

        # Mutate the genes within this segment
        for i in range(start_idx, end_idx):
            if random() < mutation_probability:
                genome[i] = randint(1, Num_rows)  # Randomly mutate to a new row number

        # Move to the next segment
        start_idx = end_idx

    return genome


def path_from_genome(genome: List[int], num_col: int, goal: Tuple[int, int] = (7, 19)) -> List[Tuple[int, int]]:
    path = []
    k = 1  # Column tracker

    # Process the genome to create the path
    for i in range(num_col - 1):
        if genome[i] < genome[i + 1]:
            for j in range(genome[i], genome[i + 1] + 1):
                path.append((j, k))
        else:
            for j in range(genome[i], genome[i + 1] - 1, -1):
                path.append((j, k))
        k += 1

        # Ensure path stays within bounds rows and cols
        if not (1 <= path[-1][0] <= Num_rows and 1 <= path[-1][1] <= Num_col):
            break

    return path


# Evolution loop for implement the Algorithm
def run_evolution(population: Population, generations: int) -> Tuple[Genome, int]:
    for generation in range(generations):

        fitness_scores = fitness_function(population, m)[0]
        infeasible_steps = fitness_function(population, m)[1]
        best_genome_idx = fitness_scores.index(max(fitness_scores))
        print(f"Generation: {generation}, (Best Genome) Fitness: {fitness_scores[best_genome_idx]}, Infeasible steps: {infeasible_steps[best_genome_idx]}")

        # Choose the best solution according to the fitness and infeasible step
        if infeasible_steps[best_genome_idx] == 0:
            if fitness_scores[best_genome_idx] >= TARGET_FITNESS:
                return population[best_genome_idx], generation

        # Sort by fitness
        population = [genome for _, genome in sorted(zip(fitness_scores, population), reverse=True)]
        # Keep top 2
        next_generation = population[:2]


        # Crossover and mutation
        for _ in range((POPULATION_SIZE - 2) // 2):
            parent1, parent2 = population[0], population[1]
            child1, child2 = mod7_crossover(parent1, parent2)

            next_generation.append(segmented_mutation(child1))
            next_generation.append(segmented_mutation(child2))


        population = next_generation
        print(population[:2])

    return None, generations

def main():

    # Run the genetic algorithm
    initial_population = generate_population(POPULATION_SIZE, Num_col, Num_rows)
    solution, generations = run_evolution(initial_population, MAX_GENERATIONS)
    print("initial_population(4):", initial_population[:4])
    
    # Display results
    if solution:

            solution_path = path_from_genome(solution, Num_col+1)
            print(f"Solution found in generation, {generations}, solution, {solution}")

            # Get fitness and infeasible steps for the best solution
            fitness_scores = fitness_function([solution], m)  # Pass solution as a population of one
            best_fitness = fitness_scores[0]

            print("Fitness:", best_fitness)
            print("Infstep:", fitness_scores[1])

            # Generate the best solution path, ensuring it reaches the goal at (7, 19)
            best_solution = solution_path + [GOAL]

            print("Solution path:", best_solution)
            # Plot maze and solution path
            a = agent(m, START[0], START[1], filled = True, footprints=True, color='red')
            m.tracePath({a: best_solution})
            m.run()
            print("Solution found in Generation: ",generations)

            return True, generations
    else:
            print("No solution found. Try again (Re-generate the initial_population)")
            print(generations)
            return False, generations


In [9]:
main()

Generation: 0, (Best Genome) Fitness: 21.7, Infeasible steps: 14
[[1, 2, 1, 1, 3, 4, 4, 4, 1, 1, 6, 4, 4, 3, 4, 4, 4, 1, 1, 5, 1, 7], [1, 4, 4, 3, 1, 1, 1, 2, 6, 4, 4, 4, 4, 6, 2, 2, 2, 1, 4, 6, 3, 7]]
Generation: 1, (Best Genome) Fitness: 31.700000000000003, Infeasible steps: 12
[[1, 2, 1, 1, 3, 4, 4, 4, 1, 1, 6, 4, 4, 3, 4, 4, 4, 4, 1, 5, 1, 7], [1, 2, 1, 1, 3, 4, 4, 4, 1, 1, 6, 4, 4, 3, 4, 4, 4, 3, 1, 5, 1, 7]]
Generation: 2, (Best Genome) Fitness: 71.1, Infeasible steps: 3
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7]]
Generation: 3, (Best Genome) Fitness: 71.1, Infeasible steps: 3
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7]]
Generation: 4, (Best Genome) Fitness: 71.1, Infeasible steps: 3
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

(True, 1755)